In [35]:
# Need to install albumentations. Run in console
# pip install albumentations

In [1]:
import os
import random
import pandas as pd
import cv2
import albumentations as A
import numpy as np

C:\Users\hetvi\Downloads\deepfish-detection\venv\Lib\site-packages\albumentations\__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.16 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [3]:
img_folder='./datasets/images/train/'

In [4]:
# getting the name of all images
images = [img for img in os.listdir(img_folder) if img.endswith(".jpg")]
images.sort()

In [10]:
# def generate_transformation_dataframe(img):
#     """
#     Assigns a certain percent of keys to each data transformation.
#     This function is to look up the specific augmentation applied to each image.
#     It ensures all images in the same video are augmented the same way.
#     :param keys: the keys of videos that group images
#     :return: DataFrame with transformation parameters
#     """
#     df = pd.DataFrame(index=img)
#
#     # Flip horizontally only 30% of keys -> setting it to true
#     df['flipping_horizontal'] = [random.random() < 0.3 for _ in img]
#
#     # Flip vertically only 30% of keys -> setting it to true
#     df['flipping_vertical'] = [random.random() < 0.3 for _ in img]
#
#     # Rotate: Assigning a random angle between 0 and 90 degrees for 20% of keys
#     rotation_values = [random.randint(0, 90) if random.random() < 0.2 else 0 for _ in img]
#     df['rotation'] = rotation_values
#
#     # Scale: a random scale factor between 0.5 and 1.5 for 10% of keys
#     scale_values = [round(random.uniform(0.5, 1.5), 2) if random.random() < 0.1 else 1.0 for _ in img]
#     df['scaling'] = scale_values
#
#     # Brightness: Random value adjustment between -0.2 and 0.2 for 15% of keys
#     brightness_values = [round(random.uniform(-0.5, 0.2), 2) if random.random() < 0.15 else 0 for _ in img]
#     df['brightness'] = brightness_values
#
#     # Contrast: Random value adjustment between 0.5 and 1.5 for 15% of keys
#     contrast_values = [round(random.uniform(0.5, 1.2), 2) if random.random() < 0.15 else 1.0 for _ in img]
#     df['contrast'] = contrast_values
#
#     # Noise injections: Random value adjustment between 0 and 0.1 for 10% of keys
#     noise_values = [round(random.uniform(0, 0.1), 2) if random.random() < 0.1 else 0 for _ in img]
#     df['noise_injections'] = noise_values
#
#     return df
#
# transformations_df = generate_transformation_dataframe(images)
#
# print(transformations_df)
# transformations_df.to_csv('transformations_df.csv')
# # #
transformations_df = pd.read_csv('transformations_df.csv', index_col=0)
print(transformations_df)

                                                    flipping_horizontal  \
7117_Caranx_sexfasciatus_juvenile_f000000_jpg.r...                 True   
7117_Caranx_sexfasciatus_juvenile_f000001_jpg.r...                 True   
7117_Caranx_sexfasciatus_juvenile_f000002_jpg.r...                 True   
7117_Caranx_sexfasciatus_juvenile_f000003_jpg.r...                False   
7117_Caranx_sexfasciatus_juvenile_f000005_jpg.r...                 True   
...                                                                 ...   
9908_no_fish_2_f000032_jpg.rf.efda0cb412793d29b...                False   
9908_no_fish_2_f000036_jpg.rf.51389c8ed3032bf9d...                False   
9908_no_fish_2_f000037_jpg.rf.0df27c45d6a2024ac...                False   
9908_no_fish_2_f000039_jpg.rf.299166e86d898ba13...                False   
9908_no_fish_2_f000041_jpg.rf.840769ac17b056b2f...                False   

                                                    flipping_vertical  \
7117_Caranx_sexfasciatus_j

In [24]:
# Function to get augmentations for each key based on the transformations DataFrame
def get_augmentations_for_key(img, transformations_df):
    """
    To obtain the transformations for each image based on its key
    :param key: Key for the transformations (image name)
    :param transformations_df: DataFrame containing transformation parameters
    :return: transform object with the transformations specified according to the transformations_df
    """
    row = transformations_df.loc[img]
    augmentations = []

    if row['flipping_horizontal']:
        augmentations.append(A.HorizontalFlip(p=1))

    if row['flipping_vertical']:
        augmentations.append(A.VerticalFlip(p=1))

    if row['rotation'] != 0:
        augmentations.append(A.Rotate(limit=(row['rotation'], row['rotation']), p=1))

    if row['scaling'] != 1.0:
        augmentations.append(A.Affine(scale=row['scaling'], p=1))

    if row['brightness'] != 0 or row['contrast'] != 0:
        augmentations.append(A.RandomBrightnessContrast(
            brightness_limit=(row['brightness'], row['brightness']),
            contrast_limit=(row['contrast'], row['contrast']),
            p=1
        ))

    if row['noise_injections']:
        augmentations.append(A.GaussNoise(var_limit=(10.0, 50.0), p=1))

    # Final augmentation pipeline
    transform = A.Compose(
        augmentations,
        bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels'], min_visibility=0.1)
    )

    return transform

# Function to apply augmentations to an image and its bounding boxes
def augment_image_and_bboxes(image, bboxes, img, transformations_df):
    """
    Apply augmentations to an image based on key transformations and adjust bounding boxes.

    :param image: Image as a numpy array.
    :param bboxes: List of bounding boxes in YOLO format
    :param key: Key for the group (from transformations_df).
    :param transformations_df: transformations for each key
    :return: Augmented image and adjusted bounding boxes.
    """

    # Get augmentations for the key using the function defined before
    transform = get_augmentations_for_key(img, transformations_df)

    # Need to separate class ID and coordinates to obtain in Albumentations format
    albumentations_bboxes = [
        [bbox[1], bbox[2], bbox[3], bbox[4]] for bbox in bboxes
    ]
    class_labels = [bbox[0] for bbox in bboxes]  # Class IDs

    augmented = transform(image=image, bboxes=albumentations_bboxes, class_labels=class_labels)

    augmented_image = augmented['image']
    augmented_bboxes = [
        [class_labels[i], *aug_bbox] for i, aug_bbox in enumerate(augmented['bboxes'])
    ]

    return augmented_image, augmented_bboxes

# Function to read YOLO labels from text files
def read_yolo_labels(label_path):
    """
    Reads a YOLO format bounding box label from the txt files and returns as a list of bounding boxes

    :param label_path: Path to the .txt file containing YOLO format bounding boxes.
    :return: A list of bounding boxes in the format [class_id, x_center, y_center, width, height].
    """
    bboxes = []
    with open(label_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            class_id = int(parts[0])
            x_center, y_center, width, height = map(float, parts[1:])
            bboxes.append([class_id, x_center, y_center, width, height])
    return bboxes

def save_bboxes_to_txt(bboxes, output_path):
    """
    Saves bounding boxes to a text file in YOLO format.

    :param bboxes: List of bounding boxes in the format [class_id, x_center, y_center, width, height].
    :param output_path: Path to the output .txt file.
    """
    with open(output_path, 'w') as f:
        for bbox in bboxes:
            f.write(" ".join(map(str, bbox)) + "\n")

# Creating output folders for augmented images and changed bounding boxes if they don't exist
output_folder = 'datasets/images/train_aug_A/'
output_txt = 'datasets/labels/train_aug_A/'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    print(f"Output folder '{output_folder}' created.")
if not os.path.exists(output_txt):
    os.makedirs(output_txt)
    print(f"Output folder '{output_txt}' created.")

# Augmenting each image in the folder
i = 1
for img in images:
    # Getting directory path + image name to load it for transformation
    image_path = os.path.join('./datasets/images/train/', img)
    image = cv2.imread(image_path)

    # Obtaining the bounding box file from the labels/train folder
    box_replace = img.replace('.jpg', '.txt')
    box_path = os.path.join('./datasets/labels/train/', box_replace)
    bboxes = read_yolo_labels(box_path)

    # saving original images as well if there was augmentation
    row = transformations_df.loc[img]

    if list(row) == [False, False, 0, 1, 0, 1, 0]:
        output_path = os.path.join(output_folder, img)
        output_box = os.path.join(output_txt, box_replace)
        cv2.imwrite(output_path, image)
        save_bboxes_to_txt(bboxes, output_box)

    else:
        # Applying the augmentation function
        augmented_image, augmented_bboxes = augment_image_and_bboxes(image, bboxes, img, transformations_df)

        # Generate new filename for the augmented image by appending '_aug'
        filename, ext = os.path.splitext(img)
        aug_filename = filename + '_aug' + ext

        # Saving the augmented image and bounding boxes in their respective output folders
        output_path_aug = os.path.join(output_folder, aug_filename)
        aug_box_replace = box_replace.replace('.txt', '_aug.txt')
        output_box_augmented = os.path.join(output_txt, aug_box_replace)

        # Save the augmented image and bounding boxes
        cv2.imwrite(output_path_aug, augmented_image)
        save_bboxes_to_txt(augmented_bboxes, output_box_augmented)

        # Save original image and bounding box as well
        output_path = os.path.join(output_folder, img)
        output_box = os.path.join(output_txt, box_replace)
        cv2.imwrite(output_path, augmented_image)
        save_bboxes_to_txt(bboxes, output_box)
    # Convert bounding boxes back to YOLO format and save them
    with open(output_box, 'w') as f:
        for bbox in augmented_bboxes:
            f.write(" ".join(map(str, bbox)) + "\n")

    print(f"Processed {i}: {img}")
    i += 1


Output folder 'datasets/images/train_aug_A/' created.
Output folder 'datasets/labels/train_aug_A/' created.
Processed 1: 7117_Caranx_sexfasciatus_juvenile_f000000_jpg.rf.4d67296de04ef75f519c841c5ae90c14.jpg
Processed 2: 7117_Caranx_sexfasciatus_juvenile_f000001_jpg.rf.0fa1fb7ce55b3edb3b919808392d352d.jpg
Processed 3: 7117_Caranx_sexfasciatus_juvenile_f000002_jpg.rf.2f332423ed6043b39183560da27120ac.jpg
Processed 4: 7117_Caranx_sexfasciatus_juvenile_f000003_jpg.rf.12fba1172037f72cdb6e6bedb942daa5.jpg
Processed 5: 7117_Caranx_sexfasciatus_juvenile_f000005_jpg.rf.725a8a159acdfdcd16c9a91c6e9e91d5.jpg
Processed 6: 7117_Caranx_sexfasciatus_juvenile_f000006_jpg.rf.f0fd9a375a59605054736d8f23579cde.jpg
Processed 7: 7117_Caranx_sexfasciatus_juvenile_f000007_jpg.rf.a738cc3a8abb502ab1ad5b3db0409f97.jpg
Processed 8: 7117_Caranx_sexfasciatus_juvenile_f000008_jpg.rf.2692931d39eecf322e3fd7a80f083af8.jpg
Processed 9: 7117_Caranx_sexfasciatus_juvenile_f000012_jpg.rf.2988cdb56b01e0e9a4a1106aee49cb44.jpg
P

C:\Users\hetvi\Downloads\deepfish-detection\venv\Lib\site-packages\albumentations\core\composition.py:191: UserWarning: Got processor for bboxes, but no transform to process it.
  self._set_keys()


Processed 22: 7117_Caranx_sexfasciatus_juvenile_f000031_jpg.rf.8e77a78dd201f0e3f0baf5a9fbf11dba.jpg
Processed 23: 7117_Caranx_sexfasciatus_juvenile_f000032_jpg.rf.ecd8c8db8cf7003ed3314ab64327a09c.jpg
Processed 24: 7117_Caranx_sexfasciatus_juvenile_f000033_jpg.rf.74e0ed337f0dafc1bc689c902aab6aff.jpg
Processed 25: 7117_Caranx_sexfasciatus_juvenile_f000034_jpg.rf.4af2e6ace1a1f33063543bf5b54b7912.jpg
Processed 26: 7117_Caranx_sexfasciatus_juvenile_f000036_jpg.rf.7f0cdcd00d6a575871e56187b4f0d0b5.jpg
Processed 27: 7117_Caranx_sexfasciatus_juvenile_f000037_jpg.rf.7af1b00e3154792c7b4f98d32b1289eb.jpg
Processed 28: 7117_Caranx_sexfasciatus_juvenile_f000038_jpg.rf.5a9baa14be62b36ab6c6b5b0feccc893.jpg
Processed 29: 7117_Caranx_sexfasciatus_juvenile_f000039_jpg.rf.3775aeb83adc07e11e5a024446393c8e.jpg
Processed 30: 7117_Caranx_sexfasciatus_juvenile_f000040_jpg.rf.328f0dcc53f77bcdf3b6eaa61b510a9e.jpg
Processed 31: 7117_Caranx_sexfasciatus_juvenile_f000041_jpg.rf.9bd719ac008b82ea784606665deb9a25.jpg
